In [12]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm 
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
import pickle
import requests

### Question 1

In [2]:
!pipenv --version

pipenv, version 2024.2.0


### Question 2

In [6]:
!pipenv lock
!pipenv install

print('sha256:03b6158efa3faaf1feea3faa884c840ebd61b6484167c711548fce208ea09445')

Locking [packages] dependencies...
Building requirements...
Resolving dependencies....
✔ Success! Locking packages...
⠙ Locking packages...
Locking [dev-packages] dependencies...
Updated Pipfile.lock (a49030dd321cf8eda3dc2dcf6c47ff45bc056aedfb95cc43ff4a3810d0129132)!
To activate this project's virtualenv, run pipenv shell.
Alternatively, run a command inside the virtualenv with pipenv run.
Installing dependencies from Pipfile.lock (129132)...
sha256:03b6158efa3faaf1feea3faa884c840ebd61b6484167c711548fce208ea09445


### Model

In [7]:
PREFIX='https://raw.githubusercontent.com/DataTalksClub/machine-learning-zoomcamp/master/cohorts/2024/05-deployment/homework'
!wget $PREFIX/model1.bin -P '../model'
!wget $PREFIX/dv.bin -P '../model'

--2024-10-28 18:37:47--  https://raw.githubusercontent.com/DataTalksClub/machine-learning-zoomcamp/master/cohorts/2024/05-deployment/homework/model1.bin
Résolution de raw.githubusercontent.com (raw.githubusercontent.com)… 2606:50c0:8001::154, 2606:50c0:8003::154, 2606:50c0:8002::154, ...
Connexion à raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8001::154|:443… connecté.
requête HTTP transmise, en attente de la réponse… 200 OK
Taille : 850 [application/octet-stream]
Sauvegarde en : « ../model/model1.bin »

model1.bin          100%[===================>]     850  --.-KB/s    ds 0s      

2024-10-28 18:37:47 (101 MB/s) — « ../model/model1.bin » sauvegardé [850/850]

--2024-10-28 18:37:48--  https://raw.githubusercontent.com/DataTalksClub/machine-learning-zoomcamp/master/cohorts/2024/05-deployment/homework/dv.bin
Résolution de raw.githubusercontent.com (raw.githubusercontent.com)… 2606:50c0:8001::154, 2606:50c0:8003::154, 2606:50c0:8002::154, ...
Connexion à raw.githubuser

### Question 3

In [10]:
def load(filename: str):
    with open(filename, "rb") as f_in:
        return pickle.load(f_in)

dv = load("../model/dv.bin")
model = load("../model/model.bin")

client = {"job": "management", "duration": 400, "poutcome": "success"}

X = dv.transform([client])
y_pred = model.predict_proba(X)[0, 1]

print(f"Probability: {y_pred:.3f}")

Probability: 0.759


/Users/florianbaudin/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator DictVectorizer from version 1.5.2 when using version 1.5.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/florianbaudin/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator LogisticRegression from version 1.5.2 when using version 1.5.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


### Question 4

In [11]:
!pipenv install flask
!pipenv install gunicorn

To activate this project's virtualenv, run pipenv shell.
Alternatively, run a command inside the virtualenv with pipenv run.
Installing flask...
✔ Installation Succeeded
Building requirements...
Resolving dependencies....
✔ Success! Locking packages...
⠇ Locking packages...
Building requirements...
Resolving dependencies....
✔ Success! Locking packages...
⠸ Locking packages...
To activate this project's virtualenv, run pipenv shell.
Alternatively, run a command inside the virtualenv with pipenv run.
Installing dependencies from Pipfile.lock (cbe80d)...
All dependencies are now up-to-date!
Installing dependencies from Pipfile.lock (cbe80d)...
To activate this project's virtualenv, run pipenv shell.
Alternatively, run a command inside the virtualenv with pipenv run.
Installing gunicorn...
✔ Installation Succeeded
Building requirements...
Resolving dependencies....
✔ Success! Locking packages...
⠹ Locking packages...
Building requirements...
Resolving dependencies....
✔ Success! Locking p

In [23]:
url = "http://localhost:9696/predict"
client = {"job": "student", "duration": 280, "poutcome": "failure"}
requests.post(url, json=client).json()

{'get_subscription': False, 'get_subscription_proba': 0.335}

### Question 5

In [21]:
!docker images

REPOSITORY              TAG           IMAGE ID       CREATED      SIZE
svizor/zoomcamp-model   3.11.5-slim   975e7bdca086   9 days ago   130MB


### Question 6

In [25]:
url = "http://localhost:9696/predict"
client = {"job": "management", "duration": 400, "poutcome": "success"}
requests.post(url, json=client).json()

{'get_subscription': True, 'get_subscription_proba': 0.759}